In [ ]:
!pip install transformers==4.33.1 peft==0.11.1 datasets
!pip install bitsandbytes
!pip install gradio

In [ ]:
!pip install datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments,
import os,torch, wandb, platform, gradio, warnings
from datasets import load_dataset
from huggingface_hub import notebook_login

In [12]:
notebook_login()


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")


In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'


notebook_login()


In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"

dataset_name = "multi_nli"

dataset = load_dataset(dataset_name)
model_name = "meta-llama/Meta-Llama-3-8B"

dataset_name = "multi_nli"

dataset = load_dataset(dataset_name)


bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)



model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config= bnb_config,
    device_map="auto",
)

model = prepare_model_for_kbit_training(model)



In [15]:

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### zero shot

In [11]:
from datasets import load_dataset
import torch

def zero_shot_prompt(premise, hypothesis):
    prompt = (
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
        f"Does the hypothesis follow from the premise?\n"
        f"Answer (yes, no, maybe):"
    )
    return prompt

def extract_answer(output_text):
    possible_answers = ["yes", "no", "maybe"]
    for word in output_text.split():
        if word.lower() in possible_answers:
            return word.lower()
    return "no_ans"  

def predict_nli(model, tokenizer, premise, hypothesis, temperature=0.5):
    prompt = zero_shot_prompt(premise, hypothesis)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=inputs['input_ids'].shape[1] + 10,
        temperature=temperature,
        do_sample=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = extract_answer(answer.split("Answer (yes, no, maybe):")[-1].strip())
    return answer

def evaluate_model(model, tokenizer, dataset, max_samples=50):
    correct = 0
    total = 0

    label_map = {0: "yes", 2: "no", 1: "maybe"}

    for example in dataset['validation_matched']:
        if total >= max_samples:
            break
        
        premise = example['premise']
        hypothesis = example['hypothesis']
        gold_label = example['label']
        
        if gold_label not in label_map:
            continue

        predicted_label = predict_nli(model, tokenizer, premise, hypothesis)
        
        if predicted_label == label_map[gold_label]:
            correct += 1

        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy


accuracy = evaluate_model(model, tokenizer, dataset)
print(f"Accuracy: {accuracy:.2f}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Accuracy: 0.14


### one shot

In [12]:
from datasets import load_dataset

one_shot_example = (
    "A man inspects the uniform of a figure in some East Asian country.", 
    "The man is looking at something.",  
    "yes" 
)

# Function to create a one-shot prompt
def one_shot_prompt(premise, hypothesis, example_premise, example_hypothesis, example_label):
    prompt = (
        f"Example:\n"
        f"Premise: {example_premise}\n"
        f"Hypothesis: {example_hypothesis}\n"
        f"Label: {example_label}\n\n"
        f"Now, predict the label for the following example:\n"
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
        f"Does the hypothesis follow from the premise?\n"
        f"Answer (yes, no, maybe):"
    )
    return prompt

# Function to extract answer
def extract_answer(output_text):
    possible_answers = ["yes", "no", "maybe"]
    for word in output_text.split():
        if word.lower() in possible_answers:
            return word.lower()
    return "no_ans"  # Default to 'no_ans' if no valid answer is found

def predict_nli(model, tokenizer, premise, hypothesis, example, temperature=0.5):
    example_premise, example_hypothesis, example_label = example
    prompt = one_shot_prompt(premise, hypothesis, example_premise, example_hypothesis, example_label)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=inputs['input_ids'].shape[1] + 10,
        temperature=temperature,
        do_sample=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = extract_answer(answer.split("Answer (yes, no, maybe):")[-1].strip())
    return answer

# Function to evaluate the model with one-shot learning
def evaluate_model(model, tokenizer, dataset, one_shot_example, max_samples=50):
    correct = 0
    total = 0

    label_map = {0: "yes", 2: "no", 1: "maybe"}

    for example in dataset['validation_matched']:
        if total >= max_samples:
            break
        
        premise = example['premise']
        hypothesis = example['hypothesis']
        gold_label = example['label']

        # Skip examples with labels not in the label_map (e.g., -1 in MultiNLI)
        if gold_label not in label_map:
            continue

        predicted_label = predict_nli(model, tokenizer, premise, hypothesis, one_shot_example)
        
        if predicted_label == label_map[gold_label]:
            correct += 1

        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy

dataset = load_dataset('multi_nli')


accuracy = evaluate_model(model, tokenizer, dataset, one_shot_example)
print(f"Model accuracy with one-shot learning: {accuracy:.2f}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Model accuracy with one-shot learning: 0.40
